In [3]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
import optuna

import itertools

In [4]:
# TRAIN_START_DATE = '2025-01-23'
# TRAIN_END_DATE = '2025-02-13'
# TRADE_START_DATE = '2025-02-14'
# TRADE_END_DATE = '2025-03-10'

In [5]:
total_time_window = 59
today = datetime.datetime.today()
end_date = today.strftime('%Y-%m-%d')
start_date = (today - datetime.timedelta(days=total_time_window)).strftime('%Y-%m-%d')

In [17]:
from datetime import datetime, timedelta
df_aapl_raw = yf.download(tickers = "aapl", start=start_date, end=end_date, interval='15m')
# end_date = datetime.today().strftime('%Y-%m-%d')
# start_date = (datetime.today() - timedelta(days=20)).strftime('%Y-%m-%d')

# df_aapl_raw = yf.download(tickers = "aapl", start=start_date, end=end_date, interval="15m")

[*********************100%***********************]  1 of 1 completed


In [7]:
print(df_aapl_raw.head())

Price                           Close        High         Low        Open  \
Ticker                           AAPL        AAPL        AAPL        AAPL   
Datetime                                                                    
2025-01-27 14:30:00+00:00  226.720505  227.380005  224.000000  224.029999   
2025-01-27 14:45:00+00:00  225.660004  227.699997  225.419998  226.750000   
2025-01-27 15:00:00+00:00  225.190002  226.039993  224.820007  225.649994   
2025-01-27 15:15:00+00:00  227.292297  227.292297  225.060104  225.195007   
2025-01-27 15:30:00+00:00  227.880005  228.587601  227.089996  227.300003   

Price                        Volume  
Ticker                         AAPL  
Datetime                             
2025-01-27 14:30:00+00:00  13677489  
2025-01-27 14:45:00+00:00   4353866  
2025-01-27 15:00:00+00:00   2642756  
2025-01-27 15:15:00+00:00   2221605  
2025-01-27 15:30:00+00:00   3110234  


In [8]:
# 如果列是 MultiIndex，可以使用 droplevel 来去掉第二层（'AAPL'）
if isinstance(df_aapl_raw.columns, pd.MultiIndex):
    # 去掉第二层（'AAPL'），只保留第一个级别
    df_aapl_raw.columns = df_aapl_raw.columns.droplevel(1)

df_aapl_raw.columns = df_aapl_raw.columns.str.lower()
# 显示处理后的结果
print(df_aapl_raw.columns)
print(df_aapl_raw.head())

Index(['close', 'high', 'low', 'open', 'volume'], dtype='object', name='Price')
Price                           close        high         low        open  \
Datetime                                                                    
2025-01-27 14:30:00+00:00  226.720505  227.380005  224.000000  224.029999   
2025-01-27 14:45:00+00:00  225.660004  227.699997  225.419998  226.750000   
2025-01-27 15:00:00+00:00  225.190002  226.039993  224.820007  225.649994   
2025-01-27 15:15:00+00:00  227.292297  227.292297  225.060104  225.195007   
2025-01-27 15:30:00+00:00  227.880005  228.587601  227.089996  227.300003   

Price                        volume  
Datetime                             
2025-01-27 14:30:00+00:00  13677489  
2025-01-27 14:45:00+00:00   4353866  
2025-01-27 15:00:00+00:00   2642756  
2025-01-27 15:15:00+00:00   2221605  
2025-01-27 15:30:00+00:00   3110234  


In [9]:
df_aapl = df_aapl_raw.reset_index()  # 保留索引
print(df_aapl.columns) 
df_aapl['datetime'] = df_aapl['Datetime']
# df_aapl['date'] = df_aapl['Datetime'].dt.date
df_aapl['date'] = pd.to_datetime(df_aapl['Datetime'])
df_aapl['tic'] = "aapl"
df_aapl['day'] = df_aapl['Datetime'].dt.dayofweek 
#remove datetime
df_aapl = df_aapl.drop(['Datetime'], axis=1)
df_aapl = df_aapl[['date', 'close', 'high', 'low', 'open', 'volume', 'tic', 'day']]
if 'adjcp' not in df_aapl.columns:
    df_aapl['adjcp'] = df_aapl['close']
    # df_aapl['adjhigh'] = df_aapl['high']
    # df_aapl['adjlow'] = df_aapl['low']
    # df_aapl['adjopen'] = df_aapl['open']
print(df_aapl.columns)

Index(['Datetime', 'close', 'high', 'low', 'open', 'volume'], dtype='object', name='Price')
Index(['date', 'close', 'high', 'low', 'open', 'volume', 'tic', 'day',
       'adjcp'],
      dtype='object', name='Price')


In [10]:
print(df_aapl["date"].dtype) 

datetime64[ns, UTC]


In [11]:
print(df_aapl.head())

Price                      date       close        high         low  \
0     2025-01-27 14:30:00+00:00  226.720505  227.380005  224.000000   
1     2025-01-27 14:45:00+00:00  225.660004  227.699997  225.419998   
2     2025-01-27 15:00:00+00:00  225.190002  226.039993  224.820007   
3     2025-01-27 15:15:00+00:00  227.292297  227.292297  225.060104   
4     2025-01-27 15:30:00+00:00  227.880005  228.587601  227.089996   

Price        open    volume   tic  day       adjcp  
0      224.029999  13677489  aapl    0  226.720505  
1      226.750000   4353866  aapl    0  225.660004  
2      225.649994   2642756  aapl    0  225.190002  
3      225.195007   2221605  aapl    0  227.292297  
4      227.300003   3110234  aapl    0  227.880005  


In [12]:
INDICATORS_15M = [
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",

]

In [13]:

fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS_15M,
                     use_vix=False,
                     use_turbulence=False,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_aapl)


Successfully added technical indicators


In [14]:
# Ensure that processed['date'] is a datetime column with full timestamp:
processed['date'] = pd.to_datetime(processed['date'])

# Create a date_range with 5-minute frequency
list_dates = pd.date_range(
    start=processed['date'].min(), 
    end=processed['date'].max(), 
    freq='15min'
)

list_ticker = processed["tic"].unique().tolist()
combination = list(itertools.product(list_dates, list_ticker))

processed_full = pd.DataFrame(combination, columns=["date", "tic"])
# Merge on both "date" and "tic" (here the 'date' in processed has full time info)
processed_full = processed_full.merge(processed, on=["date", "tic"], how="left")
processed_full.sort_values(["date", "tic"], inplace=True)
processed_full.reset_index(drop=True, inplace=True)

# Fill missing values (you can use forward-fill or another method)
processed_full = processed_full.ffill()  # or .bfill() if appropriate

In [15]:
processed_full.head(100)

,date,tic,close,high,low,open,volume,day,adjcp,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma
0,2025-01-27 14:30:00+00:00,aapl,226.720505,227.380005,224.000000,224.029999,13677489.0,0.0,226.720505,227.690029,224.690479,0.000000,66.666667,100.000000,226.720505
1,2025-01-27 14:45:00+00:00,aapl,225.660004,227.699997,225.419998,226.750000,4353866.0,0.0,225.660004,227.690029,224.690479,0.000000,66.666667,100.000000,226.190254
2,2025-01-27 15:00:00+00:00,aapl,225.190002,226.039993,224.820007,225.649994,2642756.0,0.0,225.190002,227.424851,224.288823,0.000000,-100.000000,33.429405,225.856837
3,2025-01-27 15:15:00+00:00,aapl,227.292297,227.292297,225.060104,225.195007,2221605.0,0.0,227.292297,228.139152,224.292253,59.259491,93.641019,46.381710,226.215702
4,2025-01-27 15:30:00+00:00,aapl,227.880005,228.587601,227.089996,227.300003,3110234.0,0.0,227.880005,228.782543,224.314583,65.219942,151.992662,67.726246,226.548563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-01-28 14:15:00+00:00,aapl,229.940002,230.919998,228.479996,230.869995,8415555.0,0.0,229.940002,232.674018,228.055174,57.136811,13.524515,11.825563,229.531720
96,2025-01-28 14:30:00+00:00,aapl,235.029999,235.399994,230.880005,230.880005,9883384.0,1.0,235.029999,233.559419,227.887772,72.156574,164.373433,34.866651,229.735360
97,2025-01-28 14:45:00+00:00,aapl,234.710007,235.589996,233.880005,235.015594,4141161.0,1.0,234.710007,234.186229,227.882582,70.548843,184.952635,36.076714,229.913026
98,2025-01-28 15:00:00+00:00,aapl,236.559998,236.850006,234.380005,234.669998,3627421.0,1.0,236.559998,235.309076,227.455734,74.011943,207.184084,43.515651,230.142232


In [18]:
TRAIN_START_DATE = processed_full["date"].min() 
TRAIN_END_DATE = pd.to_datetime(TRAIN_START_DATE) + pd.Timedelta(days=total_time_window - 14)

TRADE_START_DATE = TRAIN_END_DATE  
TRADE_END_DATE = processed_full["date"].max() 

train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

4320
1173


In [ ]:
# train.to_csv('train_data.csv')
# trade.to_csv('trade_data.csv')

In [19]:
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader

from stable_baselines3.common.logger import configure
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

RESULTS_DIR = 'results'
TRAINED_MODEL_DIR = 'trained models'

In [20]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS_15M)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 9


In [21]:
# if_using_a2c = True
if_using_ddpg = True
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

In [22]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS_15M) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 9


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS_15M,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [ ]:
e_train_gym = StockTradingEnv(df=train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

simple params

In [ ]:
def sample_ddpg_params(trial:optuna.Trial):
  # Size of the replay buffer
  buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2) #trial.suggest_loguniform(name, low, high)
  batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
  
  return {"buffer_size": buffer_size,
          "learning_rate":learning_rate,
          "batch_size":batch_size}

all_ddpg params 

In [ ]:
# def sample_ddpg_params(trial: optuna.Trial):
#     # 定义超参数采样空间
#     buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
#     learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
#     batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
#     gamma = trial.suggest_categorical("gamma", [0.94, 0.96, 0.98])
#     tau = trial.suggest_categorical("tau", [0.08, 0.1, 0.12])
#     train_freq = trial.suggest_categorical("train_freq", [512, 768, 1024])
#     noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
#     noise_std = trial.suggest_categorical("noise_std", [0.1, 0.2, 0.3])
#     net_arch = trial.suggest_categorical("net_arch", ["small", "medium", "big"])

#     # 定义网络结构
#     net_arch = {
#         "small": [64, 64],
#         "medium": [256, 256],
#         "big": [512, 512],
#     }[net_arch]

#     return {
#         "buffer_size": buffer_size,
#         "learning_rate": learning_rate,
#         "batch_size": batch_size,
#         "gamma": gamma,
#         "tau": tau,
#         "train_freq": train_freq,
#         "noise_type": noise_type,
#         "noise_std": noise_std,
#         "policy_kwargs": dict(net_arch=net_arch)
#     }

In [ ]:
def objective(trial: optuna.Trial):
    hyperparameters = sample_ddpg_params(trial)
    agent = DRLAgent(env=env_train)
    model_ddpg = agent.get_model("ddpg", model_kwargs=hyperparameters)
    trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name="ddpg", total_timesteps=50000)
    df_account_value_ddpg, _ = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)
    print(df_account_value_ddpg.head())  # 打印前几行数据
    print(df_account_value_ddpg.dtypes)  # 打印数据类型
    
    # 提取账户价值列（假设列名为 "account_value"）
    account_values = df_account_value_ddpg["account_value"]
    cumulative_return = (account_values.iloc[-1] - account_values.iloc[0]) / account_values.iloc[0]
    # cumulative_return = (df_account_value_ddpg.iloc[-1] - df_account_value_ddpg.iloc[0]) / df_account_value_ddpg.iloc[0]
    return cumulative_return

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10) # more trials will be better

[I 2025-03-21 20:51:43,380] A new study created in memory with name: no-name-9256fced-0eb4-436a-b137-4db29e5cd207
/var/folders/kx/hqt1dvlj7_b21v_g8477jmnr0000gn/T/ipykernel_96293/2632089047.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2) #trial.suggest_loguniform(name, low, high)


{'buffer_size': 100000, 'learning_rate': 2.7535512350160737e-05, 'batch_size': 128}
Using cpu device
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 344      |
|    time_elapsed    | 22       |
|    total_timesteps | 7832     |
| train/             |          |
|    actor_loss      | 6.44e+04 |
|    critic_loss     | 2e+03    |
|    learning_rate   | 2.75e-05 |
|    n_updates       | 7731     |
|    reward          | 0.0      |
---------------------------------
day: 1957, episode: 40
begin_total_asset: 1000000.00
end_total_asset: 1000000.00
total_reward: 0.00
total_cost: 0.00
total_trades: 0
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 345      |
|    time_elapsed    | 45       |
|    total_timesteps | 15664    |
| train/             |          |
|    actor_loss      | 4.23e+04 |
|    critic_loss     | 2.57e+05 |
|    learning_rate

[I 2025-03-21 20:54:08,774] Trial 0 finished with value: 0.0 and parameters: {'buffer_size': 100000, 'learning_rate': 2.7535512350160737e-05, 'batch_size': 128}. Best is trial 0 with value: 0.0.


hit end!
                       date  account_value
0 2025-02-14 00:00:00+00:00      1000000.0
1 2025-02-14 00:15:00+00:00      1000000.0
2 2025-02-14 00:30:00+00:00      1000000.0
3 2025-02-14 00:45:00+00:00      1000000.0
4 2025-02-14 01:00:00+00:00      1000000.0
date             datetime64[ns, UTC]
account_value                float64
dtype: object
{'buffer_size': 10000, 'learning_rate': 0.000760606179290715, 'batch_size': 256}
Using cpu device


/var/folders/kx/hqt1dvlj7_b21v_g8477jmnr0000gn/T/ipykernel_96293/2632089047.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2) #trial.suggest_loguniform(name, low, high)


...

hit end!
                       date   account_value
0 2025-02-14 00:00:00+00:00  1000000.000000
1 2025-02-14 00:15:00+00:00   999975.848000
2 2025-02-14 00:30:00+00:00   999951.695999
3 2025-02-14 00:45:00+00:00   999927.543999
4 2025-02-14 01:00:00+00:00   999903.391998
date             datetime64[ns, UTC]
account_value                float64
dtype: object
{'buffer_size': 1000000, 'learning_rate': 0.0002379449250830646, 'batch_size': 32}
Using cpu device


/var/folders/kx/hqt1dvlj7_b21v_g8477jmnr0000gn/T/ipykernel_96293/2632089047.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2) #trial.suggest_loguniform(name, low, high)


day: 1957, episode: 270
begin_total_asset: 1000000.00
end_total_asset: 998018.71
total_reward: -1981.29
total_cost: 1093.69
total_trades: 98
Sharpe: -0.584
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 445       |
|    time_elapsed    | 17        |
|    total_timesteps | 7832      |
| train/             |           |
|    actor_loss      | -1.62e+04 |
|    critic_loss     | 8.3e+05   |
|    learning_rate   | 0.000238  |
|    n_updates       | 7731      |
|    reward          | 0.0       |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 441       |
|    time_elapsed    | 35        |
|    total_timesteps | 15664     |
| train/             |           |
|    actor_loss      | -1.06e+04 |
|    critic_loss     | 1.91e+03  |
|    learning_rate   | 0.000238  |
|    n_updates       | 15563     |
|   

[I 2025-03-21 21:53:46,201] Trial 9 finished with value: 0.0 and parameters: {'buffer_size': 1000000, 'learning_rate': 0.0002379449250830646, 'batch_size': 32}. Best is trial 0 with value: 0.0.


hit end!
                       date  account_value
0 2025-02-14 00:00:00+00:00      1000000.0
1 2025-02-14 00:15:00+00:00      1000000.0
2 2025-02-14 00:30:00+00:00      1000000.0
3 2025-02-14 00:45:00+00:00      1000000.0
4 2025-02-14 01:00:00+00:00      1000000.0
date             datetime64[ns, UTC]
account_value                float64
dtype: object


In [ ]:
best_params = study.best_params
print("Best hyperparameters:", best_params)
trained_ddpg = agent.get_model("ddpg", model_kwargs=best_params)
trained_ddpg = agent.train_model(model=trained_ddpg, tb_log_name="ddpg", total_timesteps=50000)

Best hyperparameters: {'buffer_size': 100000, 'learning_rate': 2.7535512350160737e-05, 'batch_size': 128}
{'buffer_size': 100000, 'learning_rate': 2.7535512350160737e-05, 'batch_size': 128}
Using cpu device
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 347      |
|    time_elapsed    | 22       |
|    total_timesteps | 7832     |
| train/             |          |
|    actor_loss      | -865     |
|    critic_loss     | 2.02e+06 |
|    learning_rate   | 2.75e-05 |
|    n_updates       | 7731     |
|    reward          | 0.0      |
---------------------------------
day: 1957, episode: 70
begin_total_asset: 1000000.00
end_total_asset: 1057258.44
total_reward: 57258.44
total_cost: 998.98
total_trades: 1957
Sharpe: 0.303
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 344       |
|    time_elapsed    | 45        |
|    total_timesteps

In [ ]:
from optuna.visualization import plot_optimization_history, plot_param_importances
plot_optimization_history(study)
plot_param_importances(study)